<a href="https://colab.research.google.com/github/nicolazilio0/deepRiccy/blob/main/Complete_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Load Dataset from local path or Google Drive </h1>

<h3> Google Drive </h3>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

! tar -zxvf /content/drive/MyDrive/refcocog.tar.gz
! pip3 install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt 
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

local_path = '/content/refcocog/images/'
local_annotations = '/content/refcocog/annotations/'

<h3> Local path </h3>

In [ ]:
local_path = './refcocog/images/' 
local_annotations = './refcocog/annotations/' 

<h1> Installations and imports </h1>

<h3> Install dependencies and modules </h3>

In [ ]:
! pip3 install ftfy regex tqdm
! pip3 install diffusers==0.11.1
! pip3 install transformers scipy ftfy accelerate
! pip3 install stanza


In [ ]:
import json
import pickle
import torch
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
import pandas as pd
from pkg_resources import packaging
import clip
import numpy as np

import os
import skimage
import IPython.display

from collections import OrderedDict
import torch

import os
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from collections import OrderedDict
import torch
import torch
from diffusers import StableDiffusionPipeline
from diffusers import DPMSolverMultistepScheduler
import stanza
from torchvision import transforms


from transformers import AutoProcessor, BlipForConditionalGeneration






<h1> Dataset Process </h1>

In [ ]:
class Coco(Dataset):
    def __init__(self, path_json, path_pickle, train=True):
        self.path_json = path_json
        self.path_pickle = path_pickle
        self.train = train

        #load images and annotations
        with open(self.path_json) as json_data:
            data = json.load(json_data)
            self.img_frame = pd.DataFrame(data['images'])
            self.ann_frame = pd.DataFrame(data['annotations'])

        #load annotations
        with open(self.path_pickle, 'rb') as pickle_data:
            data = pickle.load(pickle_data)
            self.refs_frame = pd.DataFrame(data)

        self.size = self.refs_frame.shape[0]

        #separate each sentence in dataframe
        self.refs_frame = self.refs_frame.explode('sent_ids')
        self.refs_frame = self.refs_frame.explode('sentences')
        self.refs_frame = self.refs_frame.reset_index(drop=True)

        self.dataset = pd.merge(self.refs_frame, self.img_frame, left_on='ann_id', right_on='id')
        self.dataset = self.dataset.drop(columns=['ann_id'])

    def __len__(self):
        return self.size
    
    def __getitem__(self, idx):
        return self.dataset.iloc[idx]

In [ ]:
def split_string(string):
    string = string.split("_")
    string = string[:-1]
    string = "_".join(string)
    append = ".jpg"
    string = string + append
    
    return string

In [ ]:
#test dataset

dataset = Coco(local_annotations + 'instances.json', local_annotations + "refs(umd).p")
print(dataset[0])

In [ ]:
#test clip
clip.available_models()

<h1> Extract bounding boxes with YOLO </h1>

In [ ]:
! rm bb*
#define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#pass image into yolo
yolo = torch.hub.load('ultralytics/yolov5', 'yolov5x', pretrained=True)

#input = dataset[55]
input = dataset[370]

image = split_string(input["file_name_x"])

yolo_output = yolo(local_path+image)

class_names=yolo.names
model, preprocess = clip.load("ViT-B/32")
model.eval()

#pass each bounding box segmented image into clip with the sentence
for i in range(len(yolo_output.xyxy[0])):



    # convert the tensor to a numpy array
    #numpy_array = cpu_tensor.numpy()
    #x1 = yolo_output.xyxy[0][i][0].numpy()
    class_index=int(yolo_output.pred[0][i][5])
    label=class_names[class_index]
    print(label)
    x1 = yolo_output.xyxy[0][i][0].cpu().numpy()
    x1 = np.rint(x1)

    y1 = yolo_output.xyxy[0][i][1].cpu().numpy()
    y1 = np.rint(y1)
    x2 = yolo_output.xyxy[0][i][2].cpu().numpy()
    x2 = np.rint(x2)
    y2 = yolo_output.xyxy[0][i][3].cpu().numpy()
    y2 = np.rint(y2)

    print(x1, y1, x2, y2)

    img = Image.open(local_path+image).convert("RGB")    
    img = img.crop((x1, y1, x2, y2))
    if(i<10):
      img.save("bb_0"+str(i)+".jpg")
    else:
      img.save("bb_"+str(i)+".jpg")

    %matplotlib inline
    plt.imshow(img)
    plt.show()
    img = preprocess(img).unsqueeze(0)
    image_imput = torch.tensor(img)
    sentence = input["sentences"]["raw"]
    print(sentence)
    text = clip.tokenize([sentence])
    
    ''' with torch.no_grad():
        image_features = model.encode_image(img).float()
        text_features = model.encode_text(text).float()
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T
    print(similarity)'''


<h3> Display image </h3>

In [ ]:
fnds = split_string(input["file_name_x"])

poer=Image.open(local_path+fnds)
poer

<h3> Padd YOLO bounding boxes with average pixel value </h3>

Since the bounding boxes that we obtain as output from YOLO are not in a predefined shape, and by default CLIP preprocesses them in order to have them in a size of 224x224, and feeding them into the CLIP encoder without processing them will cause them to be stretched, enlarged resulting in a loss of information. We don't want to lose information on our images, so we decided to apply a padding of the image in order to have them resized by 224x224 but mantaining also the informations. 
The colour we applied to the padding depends on the mean value of the colors of the pixels of the image. This is done in order both to reduce the loss of informaion, but also not to add noise or other informations inside the image. 
The way we apply the padding depends on the size of the image: 
- if the image has one of the two dimensions larger than 224, we create a square image with the size of the larger between width and height of the image, and with colour as the mean value of the colors. We then put our starting image at the center, and in the end it is resized to 224x224
- if both the dimensions are lower than 224, the original image is just padded

An example of the results we obtain is the following:
![picture](https://drive.google.com/uc?id=1UgdwmBkeVIV0Zz21nKdJpYZ2NEQfk229
)


In [ ]:
def padd_image(img):
  print(type(img))
  avg_color_per_row = np.average(img, axis=0)
  avg_color = np.average(avg_color_per_row, axis=0)
  old_image_width,old_image_height  = img.size
  # create new image of desired size and color (blue) for padding
  if(old_image_height>224 or old_image_width>224):
    if(old_image_height>old_image_width):
          new_image_width = old_image_height
          new_image_height = old_image_height


    else:
          new_image_width = old_image_width
          new_image_height = old_image_width

    
    color=avg_color
    #color = (255,0,255)

    result = np.full((new_image_height,new_image_width, 3), color, dtype=np.uint8)

    # compute center offset
    x_center = (new_image_width - old_image_width) // 2
    y_center = (new_image_height - old_image_height) // 2

    # copy img image into center of result image
    result[y_center:y_center+old_image_height, 
          x_center:x_center+old_image_width] = img   

        
  else:
    new_image_width = 224
    new_image_height = 224
    '''if(old_image_height>old_image_width):
          new_image_width = old_image_height
          new_image_height = old_image_height


    else:
          new_image_width = old_image_width
          new_image_height = old_image_width'''
 
    #color = (255,0,255)
    color=avg_color
    result = np.full((new_image_height,new_image_width, 3), color, dtype=np.uint8)

    # compute center offset
    x_center = (new_image_width - old_image_width) // 2
    y_center = (new_image_height - old_image_height) // 2

    # copy img image into center of result image
    result[y_center:y_center+old_image_height, 
          x_center:x_center+old_image_width] = img


  img= Image.fromarray(result)
  img=img.resize((224,224))
  return img

In [ ]:
original_images = []
images = []
texts = []
plt.figure(figsize=(16, 5))

for filename in [filename for filename in sorted(os.listdir(os.getcwd())) if filename.endswith(".png") or filename.endswith(".jpg")]:
    print(filename)
    name = os.path.splitext(filename)[0]

    image = Image.open(os.path.join(os.getcwd(), filename)).convert("RGB")
    image=padd_image(image)

  
    plt.subplot(4, 8, len(images) + 1)
    plt.imshow(image)
    plt.xticks([])
    plt.yticks([])

    original_images.append(image)
    images.append(preprocess(image))    
    texts.append(filename)


plt.tight_layout()

<h3> Build images features </h3>

In [ ]:
image_input = torch.tensor(np.stack(images)).cuda()
text_tokens = clip.tokenize(["This is " + desc for desc in texts]).cuda()

print(image_input.size())

<h1> Stanza NLP analysis </h1>

In order to discriminate our predictions we use some Natural Language Processing techniques in order to find the subject of our caption. 
What we decided to use is the Dependency graph. This is useful because it represent the dependencies between the various tokens inside of the sentence. 
In fact in each sentence we can define the so called **root** of the sentence, which is the token on which all other tokens in the sentence depend. This can generate two possible scenarios: 
- **The root is a noun**: this is the best case for our task since we can identify that name as the subject of our caption.
- **The root is a verb**:  This create what is called a "verbal phrase", which is a sentence in which the root is the verb. Removing it from the sentence would make it lose meaning. When we have such type of sentence, there is high chance of having inside of the dependencies what is called "NSUBJ" or  [Nominal Subject](https://universaldependencies.org/en/dep/nsubj.html) (both active and passive). If it is present than this is the subject of our sentence. 
Otherwise if it is not present, once we have found the root of the sentence as a verb, we go back from the verb to the beginning of the sentence and we select as root of the sentence the first noun that we find. 






<img src='https://editor.analyticsvidhya.com/uploads/29920Screenshot%20(127).png' >
Example of a dependency graph

<h3> Install stanza dependency graph model </h3>

In [ ]:
#import stanza NLP model for english language
stanza.download('en',model_dir='/models/english',package='partut')
nlp = stanza.Pipeline(lang='en')

<h3> Preprocess input caption </h3>

We noticed that there was a problem with the so called "not well-formed" sentences (*In natural language processing, a not well-formed sentence refers to a sentence that does not conform to the rules and conventions of the language, making it difficult or impossible to be accurately processed and understood by a computer program or a human reader*), this was creating the effect of not being able to find the correct root of the sentence. 
For example one of the cases we encountered was the sentence: "*boy holding two bears*" this after we analyzed it with [corenlp](https://corenlp.run/) gave us the following result: 
![picture](https://drive.google.com/uc?id=1KrY63kV1KwWWsjyyPnIlsOW1rUtU0KXM
)

This is an example of a "not well-formed" sentence, in fact although the root of the sentence is the verb "holding", when we apply the NLP parsing we cannot retrieve the noun "boy" as the subject of the sentence since it is recognised as an [INTJ](https://universaldependencies.org/u/pos/INTJ.html).

In order to solve this problem we noticed that adding a [determiner](https://universaldependencies.org/u/pos/DET.html) at the beginning of the sentence allows us to avoid having such problems. Between all the possible determiners we chose "*the*" as it does not change the meaning of the sentence and it is the most general one. 

After this is applied we obtain this dependecy parsing: 

![picture](https://drive.google.com/uc?id=1Rt4nU5kXKSn-pq6nRkaCAcyp__m7Z1cd
)
In which we can see that the word "boy" is being parsed as a [noun](https://universaldependencies.org/u/pos/NOUN.html)

In [ ]:
nlp_sent=input["sentences"]["raw"]
# put sentence in lowecase
nlp_sent=nlp_sent.lower()
# if the sentence does not start with "a" or "the" insert it
x = nlp_sent.split(" ")
if(x[0]!="the" and x[0]!="a"):
    nlp_sent="the "+nlp_sent

print(nlp_sent)

<h3> Analyze caption and compute features of its root </h3>

After we obtained the root of the sentence we use the CLIP text encoder to get the features for our root

In [ ]:

nlp_sent=nlp_sent.lower()
print(nlp_sent)
# Process the sentence
doc = nlp(nlp_sent)
# print nlp dependencies
doc.sentences[0].print_dependencies()

print(input["sentences"]["raw"])
root=''
phrase_upos=[]
# get heads of words
heads=[sent.words[word.head-1].text for sent in doc.sentences for word in sent.words]
for sent in doc.sentences:
   for word in sent.words:
     #if it is a verbal phrase then take the nominal subject of the phrase
     if(word.deprel=='nsubj' or word.deprel=='nsubj:pass'):
       root=word.text
       print(word.text)
       break
     print(word)  
     phrase_upos.append(word)
     # else take the root of the phrase
     if(word.head==0):
       print(word.text)
       root=word.text
       # if the root is a verb
       if(word.upos=='VERB'):
         for w in reversed(phrase_upos):
          # go back until you get a noun
          if(w.upos=='NN'):
             print(w.text)
   
 



#print(heads)
# this can be used to tokenize an entire phrase
'''prompt_tokens=clip.tokenize(nlp_sent,context_length=77,truncate=True).cuda()
with torch.no_grad():
  prompt_features = model.encode_text(prompt_tokens).float()
print(prompt_features.size())'''
# this is used to tokenize only the single word
prompt_tokens=clip.tokenize(root,context_length=77,truncate=True).cuda()
# calculate features
with torch.no_grad():
  prompt_features = model.encode_text(prompt_tokens).float()


<h3> Analyze YOLO class prediction </h3>

We then encode also the classes that are present inside our YOLO predictions. These encodings will be used to compute the similarity between the root of our sentence and the YOLO classes in order to bring us back to one of the original YOLO classes. Having the subject of our sentence expressed as one of the YOLO classes we can exclude all the bounding boxes that have a label that is different from the one of our subject.

In [ ]:
names=[]

# for each of the YOLO output prediction get the correspondent class
for i in range(len(yolo_output.xyxy[0])):
    class_index=int(yolo_output.pred[0][i][5])
    label=class_names[class_index]
    print(label)
    names.append(label)


# tokenize and create features for the classes
tokens=clip.tokenize(names,context_length=77,truncate=True).cuda()
with torch.no_grad():
  classes_features = model.encode_text(tokens).float()
print(classes_features.size())

<h3> Compute prompt predictions in order to check similarity between root and YOLO class </h3>

In [ ]:
# compute similarities between YOLO predicrions and the subject of the phrase
prompt_features /= prompt_features.norm(dim=-1, keepdim=True)
classes_features /= classes_features.norm(dim=-1, keepdim=True)
prompt_similarity = classes_features.cpu().numpy() @ prompt_features.cpu().numpy().T

<h3> Take class with maximum similarity </h3>

We only want to have the class with the best similarity

In [ ]:
# take as desired class the one with the highest similarity
rappresentation=np.argmax(prompt_similarity)
print(root)
interested_class=names[rappresentation]

print(interested_class)

<h1> Stable Diffusion </h1>

<h3> Import stable diffusion model and create pipeline </h3>

In [ ]:
# Load stable diffusion mode
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)  
pipe = pipe.to("cuda")


<h3> Generate images </h3>

Since we know that Stable Diffusion has a component of randomness in the image generation we deciced to take into consideration creating more than one image. We decided to create more than one image in order to reduce the weight of a possible "outlier" in the images.

We then 

In [ ]:
# insert the prompt the caption we are provided by the dataset
prompt= 'Use deep learning algorithms to generate a hyper-realistic portrait of a'+   input["sentences"]["raw"] +' Use advanced image processing techniques to make the image appear as if it were a photograph'

# create the images with stable diffusion
stable_input = pipe(prompt,num_inference_steps=50).images[0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)
stable_input2 = pipe(prompt,num_inference_steps=50).images[0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)
stable_input3 = pipe(prompt,num_inference_steps=50).images[0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)

stable_input

<h3> Create encoding for images </h3>

In [ ]:

with torch.no_grad():
    # transform image into tensor
    convert_tensor = transforms.ToTensor()
    # resize image for clip
    stable_input=stable_input.resize([224,224])   
    stable_input2=stable_input2.resize([224,224])   
    stable_input3=stable_input3.resize([224,224])   

    # create tensors
    image_stable=torch.tensor(np.stack(convert_tensor(stable_input))).cuda()   
    image_stable2=torch.tensor(np.stack(convert_tensor(stable_input2))).cuda()   
    image_stable3=torch.tensor(np.stack(convert_tensor(stable_input3))).cuda()   
    
    # stack images into a tensor
    img_tens = torch.stack([image_stable,image_stable2,image_stable3])
    
    print(image_input.size())
    print(img_tens.size())
    
    # encode stable diffusion images
    text_features= model.encode_image(img_tens).float()
    
    # encode YOLO bounding boxes
    image_features = model.encode_image(image_input).float()
    
    print(text_features.size())
    print(image_features.size())

    #stable_input=stable_input.resize([224,224])   
    


    #text_features = model.encode_text(text_tokens).float()

<h3> Compute Cosine similarity </h3>

In [ ]:
# compute cosine similarity
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T

In [ ]:
# compute mean of similarities
similarity_vec=np.mean(similarity,axis=0)


<h3> Take only the best bounding box </h3>

Once we have computed the similarity values, and obtained the mean vecotor of the similarities we want to filter out our results in order to make our prediction. The first thing we do is to filter out all the values in the similarity vector by checking if the corresponding bounding boxes have the class as the one we obtained before.
Than inside of the "filtered" vector we take the maximum value that will correspond to our prediction.

In [ ]:
possible_values=[]
indexes=[]
for i in range(len(yolo_output.xyxy[0])):
    class_index=int(yolo_output.pred[0][i][5])
    label=class_names[class_index]
    # take only the YOLO predictions that have as class the one we computed in the NLP analysis
    if(label==interested_class):
      possible_values.append(similarity_vec[i])
      indexes.append(i)
# take only the best class      
print(max(possible_values))
index=possible_values.index(max(possible_values))
index=indexes[index]
print(nlp_sent)

original_images[index]


    

<h1> Image Captioning </h1>

<h3> Import image captioning model </h3>

In [ ]:
# import image captioning model
processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model_blip = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

<h3> Create captions for YOLO bounding boxes </h3>

In [ ]:
description_images=[]
plt.figure(figsize=(16, 5))
# load the YOLO bounding boxes
for filename in [filename for filename in sorted(os.listdir(os.getcwd())) if filename.endswith(".png") or filename.endswith(".jpg")]:
    print(filename)
    name = os.path.splitext(filename)[0]
    image = Image.open(os.path.join(os.getcwd(), filename)).convert("RGB")
    plt.subplot(4, 8, len(description_images) + 1)
    plt.imshow(image)
    plt.xticks([])
    plt.yticks([])
    description_images.append(image)
    

plt.tight_layout()

In [ ]:
# process images for image captioning
inputs = processor(images=description_images, return_tensors="pt")
pixel_values = inputs.pixel_values
# generate captions
generated_ids = model_blip.generate(pixel_values=pixel_values, max_length=50)
generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)
print(generated_caption)

<h3> Create features for the generated descriptions </h3>

In [ ]:
# encode the text 
text_description = clip.tokenize(generated_caption).cuda()
text_sent = clip.tokenize([sentence]).cuda()
with torch.no_grad():

  sent_features = model.encode_text(text_sent).float()
  caption_features = model.encode_text(text_description).float()

In [ ]:
# compute similarity
caption_features /= caption_features.norm(dim=-1, keepdim=True)
sent_features /= sent_features.norm(dim=-1, keepdim=True)
similarity_text = sent_features.cpu().numpy() @ caption_features.cpu().numpy().T

<h3> Create combination of Image to Image similarity with Text to Text similarity </h3>

In [ ]:
# process the value obtained
similarity_text=similarity_text/10
similarity_vec+=similarity_text[0]


<h3> Obtain only the best value </h3>

In [ ]:
possible_values=[]
indexes=[]

for i in range(len(yolo_output.xyxy[0])):
    class_index=int(yolo_output.pred[0][i][5])
    label=class_names[class_index]
    if(label==interested_class):
      possible_values.append(similarity_vec[i])
      indexes.append(i)
print(max(possible_values))
index=possible_values.index(max(possible_values))
index=indexes[index]
print(nlp_sent)

original_images[index]